# Projeto - Visão Computacional 

<br>

#### Implementação do artigo [Predicting Biomechanical Risk Factors for Division - I Women’s Basketball Athletes](https://ieeexplore.ieee.org/document/10887636)
##### Feito por Lucas Paiva Ferruccio
---

#### • Imports

In [1121]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

res_video = (1080, 1920)
res_display = (480, 640)
res_yolo = (640,640)

video_path_frontal = "data/validate/VideoFrontal.mp4"
video_path_lateral = "data/validate/VideoLateral.mp4"

jumping = False # Variavel de controle para detectar pulos

### Funções Auxiliares:


#### • Criar vetores

In [1122]:
"""
Recebe dois pontos A e B e retorna o vetor AB
"""
def create_vector(A, B):
    return np.array(B) - np.array(A)

#### • Distância Euclidiana

In [1123]:
"""
Calcula a distancia entre os pontos A e B
"""
def dist_euclidiana(A, B):
    return np.linalg.norm(create_vector(A,B))

#### • Calculo do ponto médio

In [1124]:
def mid_point(A, B):
    A = np.array(A)
    B = np.array(B)
    
    return np.divide(np.add(A, B), 2)

#### • Calculo do angulo entre dois vetores

In [1125]:
"""
    Calcula o angulo (em Grau) entre dois vetores usando o arccos dos vetores A e B
    Ang = arccos(A.B/|A|x|B|)
"""
def calculate_angle_arrays(A, B):
    # Norma dos vetores
    mod_A = np.linalg.norm(A)
    mod_B = np.linalg.norm(B)
    
    # Produto escalar
    dot_AB = np.dot(A, B)
    
    # Calculo do angulo em radiano
    rdn = np.arccos(dot_AB / (mod_A * mod_B))
    
    # Conversão para grau
    return np.degrees(rdn)
    

### Calculo dos pontos de riscos - Visão Frontal

#### • Flexão Lateral do Tronco
 • Cálculo do grau de desvio lateral do tronco utilizando o ponto médio do quadril (23,24) e o ponto médio do ombro (11,12)

In [1126]:
def risk_lateral_trunk():    
    # Coleta as referencias de cada ponto (Ombros e quadril)    
    left_shoulder = [landmarks_frontal[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks_frontal[mp_pose.PoseLandmark.LEFT_SHOULDER].y]    
    right_shoulder = [landmarks_frontal[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, landmarks_frontal[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
    
    left_hip = [landmarks_frontal[mp_pose.PoseLandmark.LEFT_HIP].x, landmarks_frontal[mp_pose.PoseLandmark.LEFT_HIP].y]    
    right_hip = [landmarks_frontal[mp_pose.PoseLandmark.RIGHT_HIP].x, landmarks_frontal[mp_pose.PoseLandmark.RIGHT_HIP].y]
    
    
    # Mediana do ombro e quadril    
    mid_shoulder = mid_point(left_shoulder, right_shoulder)    
    mid_hip = mid_point(left_hip, right_hip)
    
    
    # Referencia central do topo    
    mid_top = np.array((0.5, 0))    
    
    # Vetores usado como base de referencia e do quadril    
    base_arr = create_vector(mid_top, mid_hip)    
    trunk_arr = create_vector(mid_shoulder, mid_hip)
    
    
    angle = calculate_angle_arrays(base_arr, trunk_arr)
    
    """
    # Adiciona os vetores na imagem
    
    # Conversão para pixelss
    mid_shoulder_px = (int(mid_shoulder[0] * res_display[0]), int(mid_shoulder[1] * res_display[1]))
    mid_hip_px = (int(mid_hip[0] * res_display[0]), int(mid_hip[1] * res_display[1]))
    mid_top_px = (int(mid_top[0] * res_display[0]), int(mid_top[1] * res_display[1]))
    
    # Desenha os vetores
    cv2.line(image_bgr_frontal, mid_top_px, mid_hip_px, (255,0,0), 1)  # Base
    cv2.line(image_bgr_frontal, mid_shoulder_px, mid_hip_px, (0,255,0), 1)  # Tronco
    """
    
    if angle > 10:    
        return True, angle
    
    return False, angle

#### • Flexão Lateral do Tronco
 • Cálculo da razão entre a distância dos ombros e a distância dos tornozelos

In [1127]:
def risk_stance_width():
    # Coleta as referencias de cada ponto (Ombros e tornozelos)    
    left_shoulder = [landmarks_frontal[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks_frontal[mp_pose.PoseLandmark.LEFT_SHOULDER].y]    
    right_shoulder = [landmarks_frontal[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, landmarks_frontal[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
    
    left_ankle = [landmarks_frontal[mp_pose.PoseLandmark.LEFT_ANKLE].x, landmarks_frontal[mp_pose.PoseLandmark.LEFT_ANKLE].y]    
    right_ankle = [landmarks_frontal[mp_pose.PoseLandmark.RIGHT_ANKLE].x, landmarks_frontal[mp_pose.PoseLandmark.RIGHT_ANKLE].y]
    
    # Calcula as distâncias
    dist_shoulders = dist_euclidiana(left_shoulder, right_shoulder)
    dist_ankles = dist_euclidiana(left_ankle, right_ankle)
    
    # Calcula a razão
    ratio = dist_shoulders / dist_ankles
    
    if ratio < 0.8:
        return "Narrow", ratio
    elif ratio > 1.2:
        return "Wide", ratio
    
    return None, ratio

#### • Simetria dos Pés na aterrissagem
 • Cálculo da diferenca de altura dos tornozelos em aterrisagem

In [1128]:
def risk_feet_symmetry():
    global jumping
    
    # Definiçao das linha de pulo
    jump_height_line = 0.88  # Altura de pulo (normalizada)
    
    # Coleta as referencias de cada ponto (Tornozelos)    
    left_heel = [landmarks_frontal[mp_pose.PoseLandmark.LEFT_HEEL].x, landmarks_frontal[mp_pose.PoseLandmark.LEFT_ANKLE].y]    
    right_heel= [landmarks_frontal[mp_pose.PoseLandmark.RIGHT_HEEL].x, landmarks_frontal[mp_pose.PoseLandmark.RIGHT_ANKLE].y]
    
    """
    # Linha de base na imagem
    cv2.line(image_bgr_frontal, (0, int(jump_height_line * res_video[1])), (res_video[0], int(jump_height_line * res_video[1])), (0,255,0), 5) # Base
    """
    
    if left_heel[1] > jump_height_line or right_heel[1] > jump_height_line:
        cv2.putText(image_bgr_frontal, f'No Chao', (10,90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2, cv2.LINE_AA)
        
        if jumping:
            heel_height_diff = abs(left_heel[1] - right_heel[1])
            
            if heel_height_diff > 0.02:
                return True, heel_height_diff
        
        jumping = False
    else:
        jumping = True
        cv2.putText(image_bgr_frontal, f'Pulando', (10,90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2, cv2.LINE_AA)
    
    
    return False, 0

### Calculo dos pontos de riscos - Visão Lateral

#### • Flexão de Joelho
 • Cálculo do angulo de flexão do joelho esquerdo.

In [1129]:
def risk_knee_flexion():
    
    # Coleta das referencias de cada ponto (Quadril, joelho e tornozelo)
    hip = [landmarks_lateral[mp_pose.PoseLandmark.LEFT_HIP].x, landmarks_lateral[mp_pose.PoseLandmark.LEFT_HIP].y]
    knee = [landmarks_lateral[mp_pose.PoseLandmark.LEFT_KNEE].x, landmarks_lateral[mp_pose.PoseLandmark.LEFT_KNEE].y]
    ankle = [landmarks_lateral[mp_pose.PoseLandmark.LEFT_ANKLE].x, landmarks_lateral[mp_pose.PoseLandmark.LEFT_ANKLE].y]
    
    # Criacção dos vetores (Coxa e Canela)
    thigh_arr = create_vector(hip, knee)
    shin_arr = create_vector(ankle, knee)
    
    # Calculo do angulo
    knee_angle = calculate_angle_arrays(shin_arr, thigh_arr) 

    """
    # Adiciona os vetores na imagem
    
    # Transforma em arrays
    hip_arr = np.array(hip)
    knee_arr = np.array(knee)
    ankle_arr = np.array(ankle)
    
    
    # Conversão para pixels
    hip_arr_px = (int(hip_arr[0] * res_display[0]), int(hip_arr[1] * res_display[1]))
    knee_arr_px = (int(knee_arr[0] * res_display[0]), int(knee_arr[1] * res_display[1]))
    ankle_arr_px = (int(ankle_arr[0] * res_display[0]), int(ankle_arr[1] * res_display[1]))
    
    # Desenha os vetores
    cv2.line(frame_resize_lateral, hip_arr_px, knee_arr_px, (255,0,0), 10)  # Coxa
    cv2.line(frame_resize_lateral, ankle_arr_px, knee_arr_px, (0,255,0), 10)  # Canela
    """
    
    if knee_angle < 30 and not jumping:
        return True, knee_angle
    
    return False, knee_angle

#### • Flexão plantar do tornozelo
 • Cálculo da amplitude do tornozelo-pé.

In [1130]:
def risk_ankle_flexion():
    # Coleta das referencias de cada ponto (Tornozelo e pé)
    foot = landmarks_lateral[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y
    heel = landmarks_lateral[mp_pose.PoseLandmark.LEFT_HEEL].y

    
    """
    # Conversão para pixels
    foot_px = (int(landmarks_lateral[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].x * res_display[0]), int(landmarks_lateral[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y * res_display[1]))
    heel_px = (int(landmarks_lateral[mp_pose.PoseLandmark.LEFT_HEEL].x * res_display[0]), int(landmarks_lateral[mp_pose.PoseLandmark.LEFT_HEEL].y * res_display[1]))
    
    # Desenha os vetores
    cv2.line(frame_resize_lateral, foot_px, heel_px, (255,0,0), 10)  # Tornozelo-Pé
    """
    
    if not jumping and heel < foot:    
        return True
    
    return False

#### • Flexão de tronco
 • Cálculo do angulo de flexão do tronco

In [1131]:
def risk_trunk_flexion():
    # Coleta das referencias de cada ponto (Ombro, quadril e joelho)
    shoulder = [landmarks_lateral[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks_lateral[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
    hip = [landmarks_lateral[mp_pose.PoseLandmark.LEFT_HIP].x, landmarks_lateral[mp_pose.PoseLandmark.LEFT_HIP].y]
    knee = [landmarks_lateral[mp_pose.PoseLandmark.LEFT_KNEE].x, landmarks_lateral[mp_pose.PoseLandmark.LEFT_KNEE].y]
    
    # Criacção dos vetores (Tronco e Coxa)
    trunk_arr = create_vector(shoulder, hip)
    thigh_arr = create_vector(hip, knee)
    
    # Calculo do angulo
    trunk_angle = calculate_angle_arrays(trunk_arr, thigh_arr) 
    
    """
    # Adiciona os vetores na imagem
    if image_bgr_lateral is not None:
        # Transforma em arrays
        shoulder_arr = np.array(shoulder)
        hip_arr = np.array(hip)
        knee_arr = np.array(knee)
        
        # Conversão para pixels
        shoulder_arr_px = (int(shoulder_arr[0] * res_display[0]), int(shoulder_arr[1] * res_display[1]))
        hip_arr_px = (int(hip_arr[0] * res_display[0]), int(hip_arr[1] * res_display[1]))
        knee_arr_px = (int(knee_arr[0] * res_display[0]), int(knee_arr[1] * res_display[1]))
        
        # Desenha os vetores
        cv2.line(frame_resize_lateral, shoulder_arr_px, hip_arr_px, (255,0,0), 10)  # Tronco
        cv2.line(frame_resize_lateral, hip_arr_px, knee_arr_px, (0,255,0), 10)  # Coxa
    """
    
    if trunk_angle > 4 and not jumping:
        return True, trunk_angle
    
    return False, trunk_angle

### • Laco principal de processamento de video

In [1132]:


"""
min_detection_confidence -> Define a porcentagem de confiança de detecção de uma pessoa
min_tracking_confidence -> Define a porcentagem de confiança de detecção dos pontos do corpo
"""

cap_frontal = cv2.VideoCapture(video_path_frontal)
cap_lateral = cv2.VideoCapture(video_path_lateral)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap_frontal.isOpened() or cap_lateral.isOpened():
        
        ret_frontal, frame_frontal = cap_frontal.read()
        ret_lateral, frame_lateral = cap_lateral.read()
        
        if ret_frontal and ret_lateral:        
            # CONVERTE para RGB (Formato do MediaPipe)
            image_frontal = cv2.cvtColor(frame_frontal, cv2.COLOR_BGR2RGB)
            image_lateral = cv2.cvtColor(frame_lateral, cv2.COLOR_BGR2RGB)
            
            # Protecao dos dados e otimizacao
            image_frontal.flags.writeable = False
            image_lateral.flags.writeable = False
            
            # Reduz o ruido na imagem
            image_frontal = cv2.GaussianBlur(image_frontal, (5,5), 0)
            image_lateral = cv2.GaussianBlur(image_lateral, (5,5), 0)
                                                             
            # PROCESSAMENTO
            results_frontal = pose.process(image_frontal)
            results_lateral = pose.process(image_lateral)
            
            
            image_frontal.flags.writeable = True
            image_bgr_frontal = cv2.cvtColor(image_frontal, cv2.COLOR_RGB2BGR)
            
            image_lateral.flags.writeable = True
            image_bgr_lateral = cv2.cvtColor(image_lateral, cv2.COLOR_RGB2BGR)
             
            # Extrai as marcações
            try:
                landmarks_frontal = results_frontal.pose_landmarks.landmark
                landmarks_lateral = results_lateral.pose_landmarks.landmark
            except:
                pass
            
            # DESENHA os landmarks na imagem RGB
            mp_drawing.draw_landmarks(image_bgr_frontal, results_frontal.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(230, 255, 0), thickness=2, circle_radius=2)
                                      )
            
            mp_drawing.draw_landmarks(image_bgr_lateral, results_lateral.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(230, 255, 0), thickness=2, circle_radius=2)
                                      )
                       
            # Analise de Riscos - visão frontal
            
            # Simetria dos pés
            
            res_feet_symmetry, height_diff = risk_feet_symmetry()
            
            if res_feet_symmetry:
                print("Risco de Simetria dos Pes detectado!")
                cv2.putText(image_bgr_frontal, f'Diferenca de Altura dos Pes: {height_diff:.2f}', 
                            (10,200), 
                            cv2.FONT_HERSHEY_SIMPLEX,
                            2, 
                            (0,0,255), 
                            2, 
                            cv2.LINE_AA)
            
            # Razao base/ombro
            
            res_stance, ratio_stance_width = risk_stance_width()
            
            if res_stance is not None:
                print("Risco de Largura de Base detectado!")
                cv2.putText(image_bgr_frontal, f'Largura de Base: {ratio_stance_width:.2f}', 
                            (10,300), 
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            2, 
                            (0,0,255), 
                            2, 
                            cv2.LINE_AA)
            
            # Flexao do tronco
            
            res_lateral_trunk, angle_lateral_trunk = risk_lateral_trunk()
            
            if res_lateral_trunk:
                print("Risco Flexao Lateral do Tronco!")
                cv2.putText(image_bgr_frontal, f'Flexao Lateral do Tronco: {angle_lateral_trunk:.2f} deg', 
                            (10,400), 
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            2, 
                            (0,0,255), 
                            2, 
                            cv2.LINE_AA)
            
            # Analise de Riscos - visão lateral
            
            # Flexao do joelho
            
            res_knee_flexion, angle_knee_flexion = risk_knee_flexion()
                
            if res_knee_flexion:
                print("Risco Flexao de Joelho detectado!")
                cv2.putText(image_bgr_lateral, f'Flexao de Joelho: {angle_knee_flexion:.2f} deg',
                            (10,200), 
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            2, 
                            (0,0,255), 
                            2, 
                            cv2.LINE_AA)
                
            # Analise de Flexao plantar dos pes
            
            res_ankle_flexion = risk_ankle_flexion()
                
            if res_ankle_flexion:
                print("Risco Flexao Plantar do Tornozelo detectado!")
                cv2.putText(image_bgr_lateral, f'Flexao Plantar do Tornozelo',
                            (10,300), 
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            2, 
                            (0,0,255), 
                            2, 
                            cv2.LINE_AA)
            
            # Analise de Flexao de Tronco
            
            res_trunk_flexion, angle_trunk_flexion = risk_trunk_flexion()
            
            if not res_trunk_flexion:
                print("Não Flexao do Tronco detectado!")
                cv2.putText(image_bgr_lateral, f'Flexao do Tronco: {angle_trunk_flexion:.2f} deg',
                            (10,400), 
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            2, 
                            (0,0,255), 
                            2, 
                            cv2.LINE_AA)
            
            
            # Redimensiona as imagens para exibição
            
            frame_resize_frontal = cv2.resize(image_bgr_frontal, res_display)
            frame_resize_lateral = cv2.resize(image_bgr_lateral, res_display)
            
            # Concatena as duas imagens
            frame_concat = cv2.hconcat([frame_resize_frontal, frame_resize_lateral])
            
            # Exibe o frame
            cv2.imshow('Pose Tracking (Pressione Q para sair)', frame_concat)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break    
        
cap_frontal.release()
cap_lateral.release()

cv2.destroyAllWindows()

Risco Flexao Plantar do Tornozelo detectado!
Não Flexao do Tronco detectado!
Não Flexao do Tronco detectado!
Risco Flexao Plantar do Tornozelo detectado!
Risco Flexao Plantar do Tornozelo detectado!
Risco Flexao Plantar do Tornozelo detectado!
Risco Flexao Plantar do Tornozelo detectado!
Risco Flexao Plantar do Tornozelo detectado!
Risco Flexao Plantar do Tornozelo detectado!
Risco Flexao Plantar do Tornozelo detectado!
Risco Flexao Plantar do Tornozelo detectado!
Não Flexao do Tronco detectado!
Risco Flexao Plantar do Tornozelo detectado!
Não Flexao do Tronco detectado!
Não Flexao do Tronco detectado!
Risco Flexao Plantar do Tornozelo detectado!
Não Flexao do Tronco detectado!
Risco Flexao Plantar do Tornozelo detectado!
Não Flexao do Tronco detectado!
Risco Flexao Plantar do Tornozelo detectado!
Não Flexao do Tronco detectado!
Risco Flexao Plantar do Tornozelo detectado!
Risco Flexao Plantar do Tornozelo detectado!
Não Flexao do Tronco detectado!
Não Flexao do Tronco detectado!
Risco

KeyboardInterrupt: 